In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# #preparing the data : Given a file, the function prepares 2 arrays
# """
# feature_set : each element is an array of 50 readings
# targets : each element is a label corrisponding to the readings with the same index in feature_set
# """
# def getSamplesFromFiles(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50):
#     targets  = np.array([])
#     feature_set= np.array([])
#     for fileName in filesNames:
#         df = pd.read_csv(fileName, header=None, delim_whitespace=True)
#         for i in range(int(df.shape[0]/readingsPerSample)):
#             if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in range(1,13):
#                 feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
#                 targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

#     feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
# #     one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
#     return feature_set[0:800], targets[0:800], feature_set[800:1000], targets[800:1000], feature_set[1000:1340], targets[1000:1340]

In [101]:
#preparing the data : Given a file, the function prepares 2 arrays
"""
feature_set : each element is an array of 50 readings
targets : each element is a label corrisponding to the readings with the same index in feature_set
"""
def getSamplesFromFiles(filesNames=["mHealth_ECGProcessed_subject3.log"],readingsPerSample=50):
    targets  = np.array([])
    feature_set= np.array([])
    for fileName in filesNames:
        df = pd.read_csv(fileName, header=None, delim_whitespace=True)
        for i in range(int(df.shape[0]/readingsPerSample)):
            if df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean() in range(1,13):
                feature_set=np.append(feature_set,[df[1][i*readingsPerSample:i*readingsPerSample+readingsPerSample]])
                targets=np.append(targets,df[2][i*readingsPerSample:i*readingsPerSample+readingsPerSample].mean()-1)

    feature_set=feature_set.reshape(int(feature_set.shape[0]/readingsPerSample),readingsPerSample)
#     one_hot_labels=one_hot_labels.reshape(int(one_hot_labels.shape[0]/12),12)
    return feature_set, targets
def unison_shuffled_copies(a, b,seed=1):
    assert len(a) == len(b)
    np.random.seed(seed)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [125]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # number of hidden nodes in each layer (512)
        hidden_1 = 50
#         hidden_2 = 33
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(100, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, 12)
#         self.fc2 = nn.Linear(hidden_1, hidden_2)
#         self.fc3 = nn.Linear(hidden_2, 12)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.6)

    def forward(self, x):
        # flatten image input
#         x = x.view(-1, 50)
#         print('XXXXXXXXXXXXX',x.shape,'\n',x)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        # add dropout layer
#         x = self.dropout(x)
        # add hidden layer, with relu activation function
#         x = self.fc3(x)        
        return x

# initialize the NN
model = Net()

In [142]:
"""
This piece of code does the following:
    1- retrieves the data from the specified files with the specified grouping number 
        (this grouping number must match the number of input nodes in the NN)
    2- shuffles the data (shuffles both the target and the feature set in the same way)
    3- separates the data into Test, Train sets given a percentage
    4- separates the Train data into Train and Val
"""

feature_set,targets=getSamplesFromFiles(["mHealth_ECGProcessed_subject3.log","mHealth_ECGProcessed_subject4.log","mHealth_ECGProcessed_subject7.log","mHealth_ECGProcessed_subject10.log"],100)
feature_set,targets=unison_shuffled_copies(feature_set,targets)
X_train, X_test, y_train, y_test = train_test_split(feature_set, targets, test_size=0.33, random_state=42)
feature_set, validation_feature_set, targets, validation_targets = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [143]:
feature_set.shape
validation_targets.shape

(180,)

In [144]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [129]:
feature_set=torch.from_numpy(feature_set).float()
targets=torch.from_numpy(targets).long()

validation_feature_set=torch.from_numpy(validation_feature_set).float()
validation_targets=torch.from_numpy(validation_targets).long()

# number of epochs to train the model
n_epochs = 100000

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
  
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(feature_set)
    # calculate the loss
    loss = criterion(output, targets)
#     print('XXXXXXXXXXXXXX',output[0])
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    # update running training loss
    train_loss = loss.item()
    
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation

    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(validation_feature_set)
    # calculate the loss
    loss = criterion(output, validation_targets)
    # update running validation loss 
    valid_loss = loss.item()
        
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss


Epoch: 1 	Training Loss: 2.491971 	Validation Loss: 2.499911
Validation loss decreased (inf --> 2.499911).  Saving model ...
Epoch: 2 	Training Loss: 2.487505 	Validation Loss: 2.499706
Validation loss decreased (2.499911 --> 2.499706).  Saving model ...
Epoch: 3 	Training Loss: 2.495937 	Validation Loss: 2.499478
Validation loss decreased (2.499706 --> 2.499478).  Saving model ...
Epoch: 4 	Training Loss: 2.487893 	Validation Loss: 2.499262
Validation loss decreased (2.499478 --> 2.499262).  Saving model ...
Epoch: 5 	Training Loss: 2.488603 	Validation Loss: 2.498989
Validation loss decreased (2.499262 --> 2.498989).  Saving model ...
Epoch: 6 	Training Loss: 2.495399 	Validation Loss: 2.498718
Validation loss decreased (2.498989 --> 2.498718).  Saving model ...
Epoch: 7 	Training Loss: 2.488120 	Validation Loss: 2.498438
Validation loss decreased (2.498718 --> 2.498438).  Saving model ...
Epoch: 8 	Training Loss: 2.474117 	Validation Loss: 2.498157
Validation loss decreased (2.49843

Epoch: 75 	Training Loss: 2.473979 	Validation Loss: 2.488647
Validation loss decreased (2.488763 --> 2.488647).  Saving model ...
Epoch: 76 	Training Loss: 2.472263 	Validation Loss: 2.488536
Validation loss decreased (2.488647 --> 2.488536).  Saving model ...
Epoch: 77 	Training Loss: 2.482255 	Validation Loss: 2.488389
Validation loss decreased (2.488536 --> 2.488389).  Saving model ...
Epoch: 78 	Training Loss: 2.474040 	Validation Loss: 2.488345
Validation loss decreased (2.488389 --> 2.488345).  Saving model ...
Epoch: 79 	Training Loss: 2.477711 	Validation Loss: 2.488196
Validation loss decreased (2.488345 --> 2.488196).  Saving model ...
Epoch: 80 	Training Loss: 2.469253 	Validation Loss: 2.488054
Validation loss decreased (2.488196 --> 2.488054).  Saving model ...
Epoch: 81 	Training Loss: 2.477071 	Validation Loss: 2.487851
Validation loss decreased (2.488054 --> 2.487851).  Saving model ...
Epoch: 82 	Training Loss: 2.474900 	Validation Loss: 2.487756
Validation loss decre

Epoch: 163 	Training Loss: 2.472899 	Validation Loss: 2.482038
Validation loss decreased (2.482086 --> 2.482038).  Saving model ...
Epoch: 164 	Training Loss: 2.473785 	Validation Loss: 2.482055
Epoch: 165 	Training Loss: 2.474881 	Validation Loss: 2.481992
Validation loss decreased (2.482038 --> 2.481992).  Saving model ...
Epoch: 166 	Training Loss: 2.471499 	Validation Loss: 2.481954
Validation loss decreased (2.481992 --> 2.481954).  Saving model ...
Epoch: 167 	Training Loss: 2.471295 	Validation Loss: 2.481862
Validation loss decreased (2.481954 --> 2.481862).  Saving model ...
Epoch: 168 	Training Loss: 2.485353 	Validation Loss: 2.481809
Validation loss decreased (2.481862 --> 2.481809).  Saving model ...
Epoch: 169 	Training Loss: 2.472471 	Validation Loss: 2.481841
Epoch: 170 	Training Loss: 2.464840 	Validation Loss: 2.481734
Validation loss decreased (2.481809 --> 2.481734).  Saving model ...
Epoch: 171 	Training Loss: 2.474538 	Validation Loss: 2.481660
Validation loss dec

Epoch: 254 	Training Loss: 2.465798 	Validation Loss: 2.478266
Validation loss decreased (2.478345 --> 2.478266).  Saving model ...
Epoch: 255 	Training Loss: 2.472060 	Validation Loss: 2.478204
Validation loss decreased (2.478266 --> 2.478204).  Saving model ...
Epoch: 256 	Training Loss: 2.465667 	Validation Loss: 2.478098
Validation loss decreased (2.478204 --> 2.478098).  Saving model ...
Epoch: 257 	Training Loss: 2.472133 	Validation Loss: 2.478090
Validation loss decreased (2.478098 --> 2.478090).  Saving model ...
Epoch: 258 	Training Loss: 2.464460 	Validation Loss: 2.478095
Epoch: 259 	Training Loss: 2.473789 	Validation Loss: 2.478118
Epoch: 260 	Training Loss: 2.468419 	Validation Loss: 2.478103
Epoch: 261 	Training Loss: 2.465396 	Validation Loss: 2.478137
Epoch: 262 	Training Loss: 2.466135 	Validation Loss: 2.478079
Validation loss decreased (2.478090 --> 2.478079).  Saving model ...
Epoch: 263 	Training Loss: 2.470169 	Validation Loss: 2.478072
Validation loss decreased

Validation loss decreased (2.475586 --> 2.475552).  Saving model ...
Epoch: 344 	Training Loss: 2.469914 	Validation Loss: 2.475540
Validation loss decreased (2.475552 --> 2.475540).  Saving model ...
Epoch: 345 	Training Loss: 2.465830 	Validation Loss: 2.475509
Validation loss decreased (2.475540 --> 2.475509).  Saving model ...
Epoch: 346 	Training Loss: 2.477671 	Validation Loss: 2.475491
Validation loss decreased (2.475509 --> 2.475491).  Saving model ...
Epoch: 347 	Training Loss: 2.468657 	Validation Loss: 2.475481
Validation loss decreased (2.475491 --> 2.475481).  Saving model ...
Epoch: 348 	Training Loss: 2.468502 	Validation Loss: 2.475483
Epoch: 349 	Training Loss: 2.472094 	Validation Loss: 2.475449
Validation loss decreased (2.475481 --> 2.475449).  Saving model ...
Epoch: 350 	Training Loss: 2.461321 	Validation Loss: 2.475487
Epoch: 351 	Training Loss: 2.456950 	Validation Loss: 2.475407
Validation loss decreased (2.475449 --> 2.475407).  Saving model ...
Epoch: 352 	T

...

Epoch: 40218 	Training Loss: 1.117796 	Validation Loss: 2.821943
Epoch: 40219 	Training Loss: 1.165157 	Validation Loss: 2.821301
Epoch: 40220 	Training Loss: 1.149635 	Validation Loss: 2.820315
Epoch: 40221 	Training Loss: 1.127139 	Validation Loss: 2.821417
Epoch: 40222 	Training Loss: 1.149474 	Validation Loss: 2.819695
Epoch: 40223 	Training Loss: 1.193632 	Validation Loss: 2.821290
Epoch: 40224 	Training Loss: 1.138012 	Validation Loss: 2.822809
Epoch: 40225 	Training Loss: 1.153938 	Validation Loss: 2.820640
Epoch: 40226 	Training Loss: 1.157643 	Validation Loss: 2.820779
Epoch: 40227 	Training Loss: 1.144207 	Validation Loss: 2.819630
Epoch: 40228 	Training Loss: 1.131135 	Validation Loss: 2.820869
Epoch: 40229 	Training Loss: 1.156960 	Validation Loss: 2.822004
Epoch: 40230 	Training Loss: 1.173297 	Validation Loss: 2.821544
Epoch: 40231 	Training Loss: 1.134715 	Validation Loss: 2.820006
Epoch: 40232 	Training Loss: 1.191756 	Validation Loss: 2.822119
Epoch: 40233 	Training Lo

Epoch: 40403 	Training Loss: 1.104630 	Validation Loss: 2.827587
Epoch: 40404 	Training Loss: 1.129580 	Validation Loss: 2.828444
Epoch: 40405 	Training Loss: 1.152188 	Validation Loss: 2.826914
Epoch: 40406 	Training Loss: 1.137639 	Validation Loss: 2.824658
Epoch: 40407 	Training Loss: 1.181665 	Validation Loss: 2.825226
Epoch: 40408 	Training Loss: 1.088036 	Validation Loss: 2.828205
Epoch: 40409 	Training Loss: 1.125624 	Validation Loss: 2.828588
Epoch: 40410 	Training Loss: 1.149676 	Validation Loss: 2.826000
Epoch: 40411 	Training Loss: 1.124491 	Validation Loss: 2.827372
Epoch: 40412 	Training Loss: 1.163194 	Validation Loss: 2.827315
Epoch: 40413 	Training Loss: 1.164988 	Validation Loss: 2.826756
Epoch: 40414 	Training Loss: 1.133300 	Validation Loss: 2.828688
Epoch: 40415 	Training Loss: 1.150575 	Validation Loss: 2.828877
Epoch: 40416 	Training Loss: 1.122659 	Validation Loss: 2.825877
Epoch: 40417 	Training Loss: 1.154821 	Validation Loss: 2.826916
Epoch: 40418 	Training Lo

Epoch: 40587 	Training Loss: 1.130722 	Validation Loss: 2.833038
Epoch: 40588 	Training Loss: 1.151999 	Validation Loss: 2.831813
Epoch: 40589 	Training Loss: 1.150455 	Validation Loss: 2.831028
Epoch: 40590 	Training Loss: 1.188011 	Validation Loss: 2.830962
Epoch: 40591 	Training Loss: 1.136318 	Validation Loss: 2.831897
Epoch: 40592 	Training Loss: 1.145967 	Validation Loss: 2.831085
Epoch: 40593 	Training Loss: 1.170175 	Validation Loss: 2.830971
Epoch: 40594 	Training Loss: 1.116556 	Validation Loss: 2.831951
Epoch: 40595 	Training Loss: 1.126704 	Validation Loss: 2.831036
Epoch: 40596 	Training Loss: 1.114104 	Validation Loss: 2.832577
Epoch: 40597 	Training Loss: 1.130705 	Validation Loss: 2.831790
Epoch: 40598 	Training Loss: 1.114370 	Validation Loss: 2.831734
Epoch: 40599 	Training Loss: 1.145345 	Validation Loss: 2.830538
Epoch: 40600 	Training Loss: 1.109709 	Validation Loss: 2.832284
Epoch: 40601 	Training Loss: 1.121432 	Validation Loss: 2.831166
Epoch: 40602 	Training Lo

Epoch: 40775 	Training Loss: 1.147343 	Validation Loss: 2.834902
Epoch: 40776 	Training Loss: 1.139922 	Validation Loss: 2.837315
Epoch: 40777 	Training Loss: 1.157999 	Validation Loss: 2.834359
Epoch: 40778 	Training Loss: 1.105872 	Validation Loss: 2.834709
Epoch: 40779 	Training Loss: 1.099703 	Validation Loss: 2.836003
Epoch: 40780 	Training Loss: 1.108933 	Validation Loss: 2.835798
Epoch: 40781 	Training Loss: 1.124225 	Validation Loss: 2.836642
Epoch: 40782 	Training Loss: 1.154536 	Validation Loss: 2.836948
Epoch: 40783 	Training Loss: 1.145365 	Validation Loss: 2.835232
Epoch: 40784 	Training Loss: 1.105227 	Validation Loss: 2.837221
Epoch: 40785 	Training Loss: 1.120767 	Validation Loss: 2.836527
Epoch: 40786 	Training Loss: 1.122500 	Validation Loss: 2.837299
Epoch: 40787 	Training Loss: 1.144269 	Validation Loss: 2.835695
Epoch: 40788 	Training Loss: 1.121211 	Validation Loss: 2.836073
Epoch: 40789 	Training Loss: 1.150987 	Validation Loss: 2.835565
Epoch: 40790 	Training Lo

KeyboardInterrupt: 

In [130]:
#Load the Model with the Lowest Validation Loss
model.load_state_dict(torch.load('model.pt'))

9

In [131]:
model.eval()
output = model(torch.from_numpy(X_test).float())

In [81]:
def accuracy(out, target):
    t = 0
    for o in range(len(out)):
        if int((output[o] == output[o].max()).nonzero()[0][0]) == int(target[o]):
            t+=1
#         print(target[o])
    print(100*t/len(out))
        

In [132]:
accuracy(output.argmax(1),y_test)


17.607223476297968


In [134]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(12))
class_total = list(0. for i in range(12))

test_feature_set=torch.from_numpy(X_test).float()
test_targets=torch.from_numpy(y_test).long()

model.eval() # prep model for evaluation


# forward pass: compute predicted outputs by passing inputs to the model 
output = model(test_feature_set)
# calculate the loss
loss = criterion(output, test_targets)
# update test loss 
test_loss = loss.item()
# convert output probabilities to predicted class
_, pred = torch.max(output, 1)
# compare predictions to true label
correct = np.squeeze(pred.eq(test_targets.data.view_as(pred)))
# calculate test accuracy for each object class
for i in range(len(test_targets)):
    label = test_targets.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

# calculate and print avg test loss
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(12):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 2.308885

Test Accuracy of     0: 30% (11/36)
Test Accuracy of     1: 15% ( 7/44)
Test Accuracy of     2: 41% (14/34)
Test Accuracy of     3: 17% ( 7/40)
Test Accuracy of     4: 19% ( 8/41)
Test Accuracy of     5:  0% ( 0/41)
Test Accuracy of     6:  5% ( 2/34)
Test Accuracy of     7:  2% ( 1/47)
Test Accuracy of     8: 15% ( 5/33)
Test Accuracy of     9: 12% ( 5/40)
Test Accuracy of    10: 41% (18/43)
Test Accuracy of    11:  0% ( 0/10)

Test Accuracy (Overall): 17% (78/443)


In [ ]:
accuracy(output,test_targets)

In [1]:
##cross validation - Doesn't work HEHEHEHE

n = len(X)
kf = KFold(n_splits=5)
fold = 0
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
#     clf = LogisticRegression().fit(X_train, y_train)
#     score = clf.score(X_test, y_test)
    m = trainModel(X_train,y_train,10000)
    m.eval() # prep model for evaluation
    y_pred = m(X_test)
    score=f1_score(y_test, y_pred.argmax(1), average='macro') 
    print("Score for fold %d: %.3f" % (fold, score))

NameError: name 'X' is not defined